In [1]:
import sys
sys.path.append('../')

from lxml import etree
import abc
import re
import requests
import pandas

from gensim.models.phrases import *

import gensim.models as models
import gensim.models.word2vec as word2vec
import secure

import searchbetter.search as search
reload(search)
import searchbetter.rewriter as rewriter
reload(rewriter)


<module 'searchbetter.rewriter' from '../searchbetter/rewriter.pyc'>

# NEW CHECKPOINT 4/17 5:18p

In [4]:
import searchbetter.rewriter as rewriter 
reload(rewriter)
import secure

model_path = secure.MODEL_PATH_BASE+'word2vec/word2vec'
w2v_rewriter = rewriter.Word2VecRewriter(model_path, create=False)

In [5]:
w2v_rewriter.rewrite("New York")

[u'philadelphia',
 u'los angeles',
 u'london',
 u'boston',
 u'chicago',
 u'princeton',
 u'seattle',
 u'chicago illinois',
 u'san francisco',
 u'ny',
 u'New York']

In [6]:
## SEARCH ENGINE DEMOS
from pprint import pprint

import searchbetter.search as search
reload(search)

import secure

# Create a search engine that searches over all edX courses.
# Under the hood, this uses Python's Whoosh library to index
# the course data stored in a CSV and then run searches against it.
dataset_path = secure.DATASET_PATH_BASE+'Master CourseListings - edX.csv'
index_path = secure.INDEX_PATH_BASE+'edx'
edx_engine = search.EdXSearchEngine(dataset_path, index_path, create=False)

In [7]:
edx_engine.search("chemistry")

[{'course_id': u'DavidsonX/D.001x/3T2014', 'name': u'Medicinal Chemistry'},
 {'course_id': u'course-v1:DavidsonX+D001x+3T2015',
  'name': u'Medicinal Chemistry'},
 {'course_id': u'course-v1:CooperUnion+Chem.1x+2T2016',
  'name': u'Preparing for CLEP Chemistry: Part 1'},
 {'course_id': u'course-v1:CooperUnion+Chem.2x+2T2016',
  'name': u'Preparing for CLEP Chemistry: Part 2'},
 {'course_id': u'MITx/3.091x_2/1T2014',
  'name': u'Introduction to Solid State Chemistry'},
 {'course_id': u'MITx/3.091x_3/3T2014',
  'name': u'Introduction to Solid State Chemistry'},
 {'course_id': u'course-v1:MITx+3.091x_4+1T2015',
  'name': u'Introduction to Solid State Chemistry'},
 {'course_id': u'course-v1:MITx+3.091x_5+3T2015',
  'name': u'Introduction to Solid State Chemistry'},
 {'course_id': u'course-v1:MITx+3.091x+3T2016',
  'name': u'Introduction to Solid State Chemistry'},
 {'course_id': u'CooperUnion/Chem.1x/1T2015',
  'name': u'Preparing for the AP* Chemistry Exam - Part 1'}]

## Testing

In [10]:
# comparison testing
search_terms = [
    "chemical reaction",
    "psychology",
    "cognitive bias",
    "machine learning"
]

rewriters = [
    rewriter.ControlRewriter(),
    rewriter.WikipediaRewriter(),
    w2v_rewriter
]

In [54]:

def stats(term):
    ans = [num_results(term, rw) for rw in rewriters]
    ans = [term] + ans
    return ans
    
    
def num_results(term, rw):
    edx_engine.set_rewriter(rw)
    results = edx_engine.search(term)
    num_results = len(results)
    return num_results

data = [stats(term) for term in search_terms]
    
#     results=  [[s['name'] for s in rr] for rr in raw_results]

In [58]:
df = pandas.DataFrame(columns=["term","control","wiki","word2vec"],data=data)

In [59]:
df

,term,control,wiki,word2vec
0,chemical reaction,0,10,6
1,psychology,10,10,58
2,cognitive bias,0,4,8
3,machine learning,10,20,10
